In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from data_loading import sine_data_generation
from utils import random_generator

EIN Batch (=128 samples) Sine Daten Generieren

In [2]:
no, seq_len, dim = 128, 24, 5 

In [3]:
data = sine_data_generation(no, seq_len, dim)

In [4]:
data[0]

array([[0.54205624, 0.54337448, 0.50690871, 0.52631528, 0.501343  ],
       [0.5859264 , 0.5916374 , 0.54355345, 0.55005655, 0.5098356 ],
       [0.62912402, 0.63903061, 0.57996366, 0.57368429, 0.51832536],
       [0.67131102, 0.68510434, 0.61594325, 0.59714494, 0.52680983],
       [0.71215719, 0.7294213 , 0.65129848, 0.62038528, 0.53528657],
       [0.75134285, 0.77156091, 0.68583896, 0.6433526 , 0.54375312],
       [0.7885613 , 0.81112323, 0.71937868, 0.66599483, 0.55220705],
       [0.82352123, 0.8477328 , 0.75173702, 0.68826061, 0.56064592],
       [0.85594902, 0.88104215, 0.78273975, 0.71009945, 0.56906729],
       [0.88559087, 0.91073517, 0.81221989, 0.73146181, 0.57746873],
       [0.91221477, 0.93653004, 0.84001871, 0.75229927, 0.58584782],
       [0.93561234, 0.95818197, 0.8659865 , 0.77256455, 0.59420214],
       [0.95560045, 0.97548546, 0.88998342, 0.7922117 , 0.60252928],
       [0.97202267, 0.98827628, 0.91188025, 0.81119617, 0.61082683],
       [0.98475045, 0.99643304, 0.

In [5]:
device = torch.device("cpu")

Klassen für einzelne Netzwerkkomponenten

In [36]:
class Time_GAN_module(nn.Module):
    """
    Class from which a module of the Time GAN Architecture can be constructed, 
    consisting of a n_layer stacked RNN layers and a fully connected layer
    
    input_size = dim of data (depending if module operates on latent or non-latent space)
    """
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Time_GAN_module, self).__init__()

        # Parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
    
            batch_size = x.size(0)

            # Initializing hidden state for first input using method defined below
            hidden = self.init_hidden(batch_size)

            # Passing in the input and hidden state into the model and obtaining outputs
            out, hidden = self.rnn(x, hidden)
        
            # Reshaping the outputs such that it can be fit into the fully connected layer
            out = out.contiguous().view(-1, self.hidden_dim)
            out = self.fc(out)
            
            # SIGMOID HINZUFÜGEN
            # HIDDEN STATES WERDEN IN DER PAPER IMPLEMENTIERUNG AUCH COMPUTED, ALLERDINGS NICHT BENUTZT?
            
            return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

Embedder & Recovery

In [8]:
x = torch.Tensor(data)

In [9]:
x.shape

torch.Size([128, 24, 5])

In [10]:
input_size = 5
output_size = 20
hidden_dim = 20
n_layers = 3

In [43]:
Embedder = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Embedder

Time_GAN_module(
  (rnn): GRU(5, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [44]:
H, _ = Embedder(x)

H = torch.reshape(H, (128, 24, 20))

H.shape

torch.Size([128, 24, 20])

In [45]:
Recovery = Time_GAN_module(input_size=H.shape[2], output_size=dim, hidden_dim=hidden_dim, n_layers=n_layers)
Recovery

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)

In [46]:
x_tilde = Recovery(H)[0]

x_tilde = torch.reshape(x_tilde, (128, 24, 5))

x_tilde.shape

torch.Size([128, 24, 5])

Generator

In [29]:
from utils import extract_time

time_information, max_seq_len = extract_time(data)

# time_information pis a list with each element the number of time points for each sample 

In [30]:
random_data = random_generator(batch_size=128, z_dim=dim, T_mb=time_information, max_seq_len=max_seq_len)

In [82]:
z = torch.tensor(random_data)
z = z.float()

z.shape

torch.Size([128, 24, 5])

In [62]:
Generator = Time_GAN_module(input_size=dim, output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers)
Generator

Time_GAN_module(
  (rnn): GRU(5, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [83]:
e_hat, _ = Embedder(z)

e_hat = torch.reshape(e_hat, (128, 24, 20))


e_hat.shape

torch.Size([128, 24, 20])

In [86]:
Supervisor = Time_GAN_module(input_size=e_hat.shape[2], output_size=hidden_dim, hidden_dim=hidden_dim, n_layers=n_layers-1)
Supervisor

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=2, batch_first=True)
  (fc): Linear(in_features=20, out_features=20, bias=True)
)

In [90]:
H_hat, _ = Supervisor(e_hat)

H_hat = torch.reshape(H_hat, (128, 24, 20))

H_hat.shape

torch.Size([128, 24, 20])

In [94]:
H_hat_supervise, _ = Supervisor(H)
   
H_hat_supervise = torch.reshape(H_hat_supervise, (128, 24, 20))    
    
H_hat_supervise.shape

torch.Size([128, 24, 20])

Recovery

In [100]:
x_hat, _ = Recovery(H_hat)

x_hat = torch.reshape(x_hat, (128, 24, 5))

x_hat.shape

torch.Size([128, 24, 5])

Discriminator

In [101]:
Discriminator = Time_GAN_module(input_size=H.shape[2], output_size=1, hidden_dim=hidden_dim, n_layers=n_layers)
Discriminator

Time_GAN_module(
  (rnn): GRU(20, 20, num_layers=3, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [105]:
Y_fake, _ = Discriminator(H_hat)

Y_fake = torch.reshape(Y_fake, (128, 24, 1))

Y_fake.shape

torch.Size([128, 24, 1])

In [106]:
Y_real, _ = Discriminator(H)

Y_real = torch.reshape(Y_real, (128, 24, 1))

Y_real.shape

torch.Size([128, 24, 1])

In [107]:
Y_fake_e, _ = Discriminator(H)

Y_fake_e = torch.reshape(Y_fake_e, (128, 24, 1))

Y_fake_e.shape

torch.Size([128, 24, 1])